In [2]:
import rasterio
from rasterio.enums import Resampling
from osgeo import gdal
from osgeo import osr
import numpy as np
import numpy.ma as ma
import pandas as pd
from scipy.ndimage import uniform_filter
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import griddata
from pykrige.rk import Krige
#from pykrige.compat import GridSearchCV
from pykrige.ok import OrdinaryKriging
from sklearn.model_selection import cross_val_score, GridSearchCV
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.mask import mask as MASK
import glob
import time
from netCDF4 import Dataset
from numpy import dtype
from datetime import date, timedelta
import random
import multiprocessing as mp
from scipy import stats

In [32]:
# In[5]:

unit = 'UX' # choose AD, UX, or L2
era5_land_ncs = glob.glob(r'/Users/brianxi/Documents/Salurbal/imputation/Results_v2/2019_Q1.nc') # can run it only on one quarter - let's try 2019 bc of the warnings
masksfile = glob.glob(r'/Users/brianxi/Documents/Salurbal/irene_ft_raster/%s/*_area_weight.tif'%unit) #change here  # choose between AD UX L2
miss_pix = gdal.Open(r'/Users/brianxi/Documents/Salurbal/imputation/ERA5land_mask.tif').ReadAsArray()
master_df = pd.DataFrame()
i=0
# read in worldpop raster file,, specifiy the exact location where you have this stored
worldpop = gdal.Open(r'/Users/brianxi/Documents/Salurbal/worldpoppers_bueno_convec.tif').ReadAsArray()
worldpop[worldpop < 0] = 0 # set no data area with 0 weight

# open the GUF urban footprint file, sepcify the exact location where you have this file stored
guf = gdal.Open(r'/Users/brianxi/Documents/Salurbal/guf_panperu_vectorized_complete.tif').ReadAsArray()
guf[guf < 0] = 0 # set no data area with 0 weight
ans = pd.DataFrame()
for maskfile in masksfile:
    SALID = maskfile.split('\\')[-1].split('_')[0]
    mask = gdal.Open(maskfile).ReadAsArray()

    # get population or GUF weight
    if SALID.startswith('105') or SALID.startswith('206'):  # decide if the area unit is in Peru or Panama
        guf_wp = guf  # if yes, use GUF as the "population" weight
    else:
        guf_wp = worldpop  # otherwise, use world pop as the weight
    mask1d = mask[mask>0]
    missing1d = miss_pix[mask>0]
    guf_wp1d = guf_wp[mask>0]*mask1d
    guf_wp1d = guf_wp1d*mask1d
    total_area = np.sum(mask1d)
    total_guf_wp1d = np.sum(guf_wp1d)
    if unit in ['AD','UX']:
        ans.loc[i,'SALID1'] = SALID # change to SALID2 if L2
    else:
        ans.loc[i, 'SALID2'] = SALID  # change to SALID2 if L2
    ans.loc[i,'%stemp_x_pct_not_imputed'%unit] = sum(mask1d[missing1d == 0])/total_area*100
    ans.loc[i,'%stemp_x_pct_imputed'%unit] = sum(mask1d[missing1d == 1])/total_area*100
    ans.loc[i, '%stemp_pw_pct_not_imputed'%unit] = sum(guf_wp1d[missing1d == 0]) / total_guf_wp1d*100
    ans.loc[i, '%stemp_pw_pct_imputed'%unit] = sum(guf_wp1d[missing1d == 1]) / total_guf_wp1d*100
    i=i+1
if unit in ['AD','UX']:
    out_name = r'/Users/brianxi/Documents/Salurbal/pct_imputed/pct_imputed_L1%s.csv'%unit
else:
    out_name = r'/Users/brianxi/Documents/Salurbal/pct_imputed/pct_imputed_%s.csv'%unit

print(out_name)
ans.to_csv(out_name) # change the unit here




/Users/brianxi/Documents/Salurbal/pct_imputed/pct_imputed_L1UX.csv
